# Car detection Project

## important imports

In [1]:
import cv2
import torch
import time
import warnings
from moviepy.editor import VideoFileClip
from PIL import Image
import numpy as np
%matplotlib inline
debug=False

## Load YOLOV5 Model

In [2]:
model = torch.hub.load('ultralytics/yolov5', "yolov5m")

Using cache found in C:\Users\10/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-5-17 Python-3.9.7 torch-1.11.0 CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 


## 1. Crop Detected Cars from frame

In [3]:
def cut_image_api(image,array_positions):
    array_photos=[]
    for single_record in array_positions:
        xmin = int(single_record[0])
        ymin = int(single_record[1])
        xmax = int(single_record[2])
        ymax = int(single_record[3])
        im1 = image[ ymin:ymax,xmin:xmax,:3]
        array_photos.append(im1)

    return array_photos


## 2.Combine Output with Cropped Cars Images

In [4]:
def pipline_outputs_combined(main_frame,stages_output):
    #Get the orginal frame height
    h1, w1 = main_frame.shape[:2]
    #Get pipeline output images size
    square_size=int(w1/len(stages_output))        
    img_3 = np.zeros((h1,w1,3), dtype=np.uint8)
    img_3[:h1, :w1,:3] = main_frame
    i=0
    #Add pipeline output images to the original frame 
    for image in stages_output:
        image=cv2.resize(image,(square_size,200),interpolation = cv2.INTER_AREA)
        img_3[0:200,i*square_size:(i*square_size)+square_size,:3] = image
        i+=1
    return img_3

## 3. Draw Detected Cars Boundries

In [5]:
def draw_prediction(img, array_out):
    for single_out in array_out:
        if (single_out[6]=='car'):
            color = (250,0,0)
            point1=(int(single_out[0]),int(single_out[1]))
            point2=(int(single_out[2]),int(single_out[3]))
            cv2.rectangle(img,point1, point2, color, 2)


## 4. Pipeline Executer

In [6]:
def detect_image(image_cv2):
    results = model(image_cv2)
    image_cv1 = image_cv2.copy()
    array_out = results.pandas().xyxy[0].to_numpy()
    cropped_car_images=cut_image_api(image_cv2,array_out)
    draw_prediction(image_cv1, array_out)
    if debug and not(len(cropped_car_images)==0):
        return pipline_outputs_combined(image_cv1,cropped_car_images)
    return image_cv1

## 5.Test Pipeline

In [11]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
debug=True
video_output ="Car_detection.mp4"
clip1 = VideoFileClip("project_video.mp4").subclip(10,20)
challenge_clip = clip1.fl_image(detect_image)
challenge_clip.write_videofile(challenge_output, audio=False)

Moviepy - Building video Car_detection.mp4.
Moviepy - Writing video Car_detection.mp4



Moviepy - Done !
Moviepy - video ready Car_detection.mp4


In [12]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))